# 🚢 LNG Cargo Diversion Decision Engine - Step by Step

## What We'll Build

You're an LNG trader with a cargo headed to **Europe (TTF)**, but **Asia (JKM)** prices are higher.

**Should you divert the cargo?**

We'll calculate:
1. **Netback** for each destination (profit after costs)
2. **Decision** (DIVERT or KEEP)
3. **Trade Ticket** (hedge sizing)
4. **Risk Analysis** (stress testing)

## Formula Reference

**Full documentation**: See `docs/formulas.md` for all formulas and sources

### Physical Constants
- LNG Density: 0.45 tonnes/m³
- Energy Content: 52 MMBtu/tonne LNG
- CO₂ Factor: 3.114 tCO₂/tonne fuel
- Boil-off Rate: 0.10% per day (industry standard for modern TFDE vessels)

### Key Formulas
- **Voyage Time**: `Days = Distance(nm) / (Speed(knots) × 24)`
- **Boil-off**: `Loss = Cargo × Rate × Days`
- **Netback**: `Revenue - Fuel Cost - Charter Cost - Carbon Cost`
- **Decision**: `Adjusted ΔNetback ≥ Threshold`

### Sources
- Vessel specs: Industry standard for 174k m³ TFDE carriers
- Market data: Platts JKM/TTF, Baltic Exchange freight
- Carbon: EU ETS (EUA) pricing

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Define Parameters

Let's set up our ship and route parameters:
- **Ship capacity**: 174,000 m³ of LNG
- **Speed**: 19.5 knots
- **Boil-off**: 0.10% per day (LNG evaporates!)
- **Routes**: US Gulf → Rotterdam (5,000 nm) vs US Gulf → Tokyo (9,500 nm)

In [7]:
# Vessel parameters
cargo_capacity_m3 = 174000       # Ship holds 174,000 cubic meters
laden_speed_kn = 19.5            # Ship speed when loaded
boil_off_pct_per_day = 0.10      # 0.10% evaporates daily
fuel_consumption_tpd = 130       # Burns 130 tonnes fuel/day

# Route distances (nautical miles)
distance_europe_nm = 5000        # US Gulf → Rotterdam
distance_asia_nm = 9500          # US Gulf → Tokyo

# Conversion constants
LNG_DENSITY = 0.45               # 1 m³ LNG = 0.45 tonnes
ENERGY_PER_TONNE = 52            # 1 tonne LNG = 52 MMBtu
CO2_FACTOR = 3.114               # 1 tonne fuel = 3.114 tonnes CO2

print(" Parameters loaded!")
print(f"   Europe distance: {distance_europe_nm:,} nm")
print(f"   Asia distance: {distance_asia_nm:,} nm")

 Parameters loaded!
   Europe distance: 5,000 nm
   Asia distance: 9,500 nm


## Calculate Voyage Time 

How long does each voyage take?

**Formula**: `Voyage Days = Distance / (Speed x 24 hours)`

In [17]:
# Calculate voyage duration
hours_per_day = 24
europe_voyage_days = distance_europe_nm / (laden_speed_kn * hours_per_day)
asia_voyage_days = distance_asia_nm / (laden_speed_kn * hours_per_day)

print(" VOYAGE TIME")
print(f"   To Europe: {europe_voyage_days:.1f} days")
print(f"   To Asia:   {asia_voyage_days:.1f} days")
print(f"   Extra time to Asia: {asia_voyage_days - europe_voyage_days:.1f} days")
print(f"\n    Asia is {((asia_voyage_days-europe_voyage_days)/europe_voyage_days)*100:.0f}% longer")

 VOYAGE TIME
   To Europe: 10.7 days
   To Asia:   20.3 days
   Extra time to Asia: 9.6 days

    Asia is 90% longer


## Calculate Boil-Off 

LNG is stored at -162°C. Even with insulation, some evaporates every day.

**Formula**: 
- `Boil-Off = Cargo × Boil-Off Rate × Voyage Days`
- `Delivered Cargo = Original Cargo − Boil-Off`

In [21]:
# Calculate boil-off for each destination
europe_boiloff_m3 = cargo_capacity_m3 * (boil_off_pct_per_day / 100) * europe_voyage_days
asia_boiloff_m3 = cargo_capacity_m3 * (boil_off_pct_per_day / 100) * asia_voyage_days

# Delivered cargo
europe_delivered_m3 = cargo_capacity_m3 - europe_boiloff_m3
asia_delivered_m3 = cargo_capacity_m3 - asia_boiloff_m3

# Convert to energy (MMBtu)
europe_delivered_mmbtu = europe_delivered_m3 * LNG_DENSITY * ENERGY_PER_TONNE
asia_delivered_mmbtu = asia_delivered_m3 * LNG_DENSITY * ENERGY_PER_TONNE

print(" BOIL-OFF LOSS")
print(f"   Europe: {europe_boiloff_m3:,.0f} m³ lost ({europe_boiloff_m3/cargo_capacity_m3*100:.2f}%)")
print(f"   Asia:   {asia_boiloff_m3:,.0f} m³ lost ({asia_boiloff_m3/cargo_capacity_m3*100:.2f}%)")

print(f"\n DELIVERED ENERGY")
print(f"   To Europe: {europe_delivered_mmbtu:,.0f} MMBtu")
print(f"   To Asia:   {asia_delivered_mmbtu:,.0f} MMBtu")
print(f"   Lost to Asia: {europe_delivered_mmbtu - asia_delivered_mmbtu:,.0f} MMBtu")

 BOIL-OFF LOSS
   Europe: 1,859 m³ lost (1.07%)
   Asia:   3,532 m³ lost (2.03%)

 DELIVERED ENERGY
   To Europe: 4,028,100 MMBtu
   To Asia:   3,988,950 MMBtu
   Lost to Asia: 39,150 MMBtu
